In [ ]:
import numpy as np
# import matplot for visualization
import matplotlib.pyplot as plt
#to see the plots inside of the notebook
%matplotlib inline
# import tensorflow and required layers
# note that tensorflow.contrib.layers was previously migrated from TF Slim.
import tensorflow as tf
from scipy import misc
from scipy.stats import entropy
import tarfile
from six.moves import urllib
import os.path
import sys

In [ ]:
model_dir='./model/'
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'


In [ ]:
def maybe_download_and_extract():
  """Download and extract model tar file."""
  dest_directory = model_dir
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Successfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)


In [ ]:
maybe_download_and_extract()

In [ ]:
def create_graph():
  """Creates a graph from saved GraphDef file and returns a saver."""
  # Creates graph from saved graph_def.pb.
  with tf.gfile.FastGFile(os.path.join(model_dir
      , 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

create_graph()

In [ ]:
def run_inference_on_image(image):
  """Runs inference on an image.

  Args:
    image: Image file name.

  Returns:
    Nothing
  """


  with tf.Session() as sess:
    # Some useful tensors:
    # 'softmax:0': A tensor containing the normalized prediction across
    #   1000 labels.
    # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
    #   float description of the image.
    # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
    #   encoding of the image.
    # Runs the softmax tensor by feeding the image_data as input to the graph.
    softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    predictions = sess.run(softmax_tensor,
                           {'DecodeJpeg:0': image})
    predictions = np.squeeze(predictions)
    # Creates node ID --> English string lookup.
    return predictions


In [ ]:
imgsNames = ['sax','bird']
imgsDir='./images/'
imgsType='.jpg'
imgs = [misc.imread(imgsDir+imgName+imgsType) for imgName in imgsNames ]
#img = misc.imread('images/sax.jpg')#,size=[600,600]#.reshape([1,600,600,3])
boxWidth=100
boxHeight=100
xStep=int(boxWidth/2)
yStep=int(boxHeight/2)


In [ ]:
def DoTheThing(img):
    #we slide on the picture with a small window, get the network ouput for the modified pic and record the KL divergence
    #between the network output of the original image and the modified image at each window
    log=dict()
    OriginalPredictions = run_inference_on_image(img)
    #copy = img.copy()
    maxEnt=0
    minEnt=1000
    for x in np.arange(0, img.shape[1], xStep):
        for y in np.arange(0, img.shape[0], xStep):
            s=img[x:x+boxWidth,y:y+boxHeight,:]
            cop=img.copy()
            cop[x:x+boxWidth,y:y+boxHeight,:]=np.zeros_like(s)
            predictions = run_inference_on_image(cop)
            #print(predictions)
            ent=entropy(OriginalPredictions,predictions)
            print('(x,y): ',(x,y),' Entropy: ',ent)
            #copy[x:x+boxWidth,y:y+boxHeight,:]=copy[x:x+boxWidth,y:y+boxHeight,:]*ent*255
            maxEnt = max(maxEnt,ent)
            minEnt = min(minEnt,ent)
            log[(x,y)]=ent
    log = {k: (v-minEnt)/(maxEnt-minEnt) for k, v in log.items()}#normalizing the divergences range to [0->1]
    return log
    #plt.imshow(copy)
    #plt.show()


In [ ]:
img =imgs[0]#sax
slog = DoTheThing(img)

In [ ]:
img = imgs[0]
copy = np.zeros_like(img)
for k,v in slog.items():
    copy[k[0]:k[0]+boxWidth,k[1]:k[1]+boxHeight,:]=copy[k[0]:k[0]+boxWidth,k[1]:k[1]+boxHeight,:]+v*100
print('sax heatmap')
plt.imshow(copy)
plt.show()

plt.imshow(img+copy)
plt.show()


In [ ]:
img = imgs[1]
blog = DoTheThing(img)

In [ ]:
img = imgs[1]
copy = np.zeros_like(img)
for k,v in blog.items():
    copy[k[0]:k[0]+boxWidth,k[1]:k[1]+boxHeight,:]=copy[k[0]:k[0]+boxWidth,k[1]:k[1]+boxHeight,:]+v*100
print('bird heatmap')
plt.imshow(copy)
plt.show()

plt.imshow(img+copy)
plt.show()


In [ ]:
# node_lookup = NodeLookup()

# top_k = predictions.argsort()[-5:][::-1]
# for node_id in top_k:
#   human_string = node_lookup.id_to_string(node_id)
#   score = predictions[node_id]
#   print('%s (score = %.5f)' % (human_string, score))